Tests for retrieving data from PACS
--

Imports
--

In [ ]:
import logging
import configparser
import pandas as pd

from datetime import datetime as dt

from IPython.core import display as ICD

from pydicom.dataset import Dataset

from scripts.run_all import run_all
from scripts.retrieve_data_from_PACS import find_data, get_data, create_dataset_from_dataframe_row

#from pynetdicom import debug_logger
#debug_logger()

pd.set_option('display.max_colwidth', -1)

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

Set the logger of the pynetdicom module to ERROR, to avoid logs

In [ ]:
logging.getLogger('pynetdicom').setLevel(logging.ERROR)

Initialize the "config" object
--

In [ ]:
config = run_all()

Get all studies for a day
--

In [ ]:
to_drop_columns = ['Query/Retrieve Level', 'Retrieve AE Title', 'Type of Patient ID', 'Issuer of Patient ID']

# create the query dataset
query_ds = Dataset()
# parameters for filtering
query_ds.QueryRetrieveLevel = 'STUDY'
query_ds.StudyDate  = '20190806'
query_ds.ModalitiesInStudy = 'PT'
# parameters to fetch
query_ds.StudyTime  = ''
query_ds.StudyInstanceUID = ''
query_ds.PatientID = ''
query_ds.StudyDescription = ''
query_ds.InstitutionName = ''
query_ds.ReferringPhysicianName = ''
# display the query dataset
logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

# do the query (C-FIND)
df_all_studies = find_data(config, query_ds)

# drop unwanted columns and display
df_all_studies = df_all_studies.drop(to_drop_columns, axis=1)
df_all_studies

Get a summary of the possible physician names
--

In [ ]:
count_phys_name = df_all_studies.groupby("Referring Physician's Name")["Referring Physician's Name"].count()
count_phys_name['None'] = len(df_all_studies[df_all_studies["Referring Physician's Name"].isnull()])
count_phys_name

Get all series for the found studies
--

In [ ]:
series_level_filters = ['Study Date', 'Patient ID']
to_drop_columns = ['Query/Retrieve Level', 'Retrieve AE Title', 'Type of Patient ID', 'Issuer of Patient ID']
sort_columns = ['Series Time', 'Number of Series Related Instances']

#for i_study in range(len(df_all_studies)):
for i_study in range(1):
    logging.info('DataFrame row:\n' + str(df_all_studies.loc[i_study, :]))
    
    # create the query dataset
    query_ds = create_dataset_from_dataframe_row(df_all_studies.loc[i_study, :], 'SERIES', incl=series_level_filters)
    # parameters for filtering
    query_ds.SeriesDate = ds.StudyDate
    query_ds.Modality = 'PT'
    # parameters to fetch
    query_ds.SeriesInstanceUID = ''
    query_ds.StudyInstanceUID = ''
    query_ds.SeriesTime = ''
    query_ds.StudyTime  = ''
    query_ds.NumberOfSeriesRelatedInstances = ''
    query_ds.SeriesDescription = ''
    # display the query dataset
    logging.info('Query Dataset:')
    for s in str(query_ds).split('\n'): logging.info('    ' + s)
    
    # do the query (C-FIND)
    df_series = find_data(config, query_ds)
    
    # drop unwanted columns, sort and display
    df_series = df_series.drop(to_drop_columns, axis=1)
    df_series.sort_values(sort_columns, inplace=True)
    ICD.display(df_series)

Go through each Series and get the first and last slice
--

In [ ]:
# create a copy of the Dataframe to be updated with the fetched values
df_series_updated = df_series.copy()

image_level_filters = ['Patient ID', 'Study Date', 'Series Instance UID', 'Modality']
to_fetch_params = ['StudyDescription', 'PatientID', 'InstitutionName', '0x07a31023', 'ReferringPhysicianName',
                        'SOPClassUID', 'ManufacturerModelName', 'InstanceNumber', 'ContentTime',
                        'AcquisitionTime', 'ActualFrameDuration', 'NumberOfSlices']

# go through each Serie
for i_serie in range(len(df_series)):
    
    logging.info('{}/{}: [{}|{}|{}|PID:{}|:{}]'.format(i_serie, len(df_series), *df_series.loc[i_serie,
                ['Study Date', 'Series Time', 'Modality', 'Patient ID', 'Series Instance UID']]))
    
    # create the query dataset
    query_ds = create_dataset_from_dataframe_row(df_series.loc[i_serie, :], 'IMAGE', incl=image_level_filters)
    # add some more filters
    query_ds.InstanceNumber = ['1', df_series.loc[i_serie, 'Number of Series Related Instances']]
    # display the Dataset
    logging.debug('Query Dataset:')
    for s in str(query_ds).split('\n'): logging.debug('    ' + s)

    # fetch the data (C-MOVE)
    df, _ = get_data(config, query_ds, to_fetch_params)
    df.reset_index(drop=True, inplace=True)
    
    # if no data is found, skip with error
    if len(df) == 0:
        logging.error('  ERROR, no data found.')
        continue
        """
        logging.error('  Problem, trying alternative indices')
        # try with a different index
        query_ds.InstanceNumber = ['2001', str(2000 + int(df_series.loc[i_serie, 'Number of Series Related Instances']))]
        # fetch the data (C-MOVE)
        df, _ = get_data(config, query_ds, to_fetch_params)
        df.reset_index(drop=True, inplace=True)
        # if no data is found, skip with error
        if len(df) == 0:
            logging.error('  ERROR, no data found.')
            continue
        """
    # if there is only one row, duplicate it
    elif len(df) == 1:
        df.loc[1, :] = df.loc[0, :]
    # if there are too many rows, skip with error
    elif len(df) > 2:
        logging.error('  ERROR, too many rows found ({}).'.format(len(df)))
        continue
       
    # copy the relevant parameters into the main DataFrame
    #df_series_updated.loc[i_serie, 'Content Time Start']     = df.loc[0, 'ContentTime']
    #df_series_updated.loc[i_serie, 'Content Time End']       = df.loc[1, 'ContentTime']
    df_series_updated.loc[i_serie, 'Acquisition Time Start'] = df.loc[0, 'AcquisitionTime']
    df_series_updated.loc[i_serie, 'Acquisition Time End']   = df.loc[1, 'AcquisitionTime']
    df_series_updated.loc[i_serie, 'Manufacturer Model Name']  = df.loc[0, 'ManufacturerModelName']
    df_series_updated.loc[i_serie, 'Actual Frame Duration']  = df.loc[0, 'ActualFrameDuration']
    df_series_updated.loc[i_serie, 'Number of Slices']  = df.loc[0, 'NumberOfSlices']
    #df_series_updated.loc[i_serie, '0x07a31023']  = df.loc[0, '0x07a31023']
    #df_series_updated.loc[i_serie, 'SOP Class UID']  = df.loc[0, 'SOPClassUID']
    
df_series_updated

In [ ]:
['2001', str(2000 + int(df_series.loc[i_serie, 'Number of Series Related Instances']))]

Trying to understand times
--

In [ ]:
df_same_times = df_series_updated[df_series_updated['Acquisition Time Start'] == df_series_updated['Acquisition Time End']]
#ICD.display(df_same_times)
df_diff_times = df_series_updated[df_series_updated['Acquisition Time Start'] != df_series_updated['Acquisition Time End']]
#ICD.display(df_diff_times)

start_stop_ranges = []
for i_serie in range(len(df_series_updated)):
    if str(df_series_updated.loc[i_serie, 'Acquisition Time Start']) == 'nan': continue
    start_stop_range = '{}-{}'.format(int(df_series_updated.loc[i_serie, 'Acquisition Time Start'].split('.')[0]),
                                      df_series_updated.loc[i_serie, 'Acquisition Time End'].split('.')[0])
    if start_stop_range not in start_stop_ranges:
        start_stop_ranges.append(start_stop_range)
      
start_stop_ranges.sort()
for r in start_stop_ranges: logging.info(r)

Get all series for a day
--

In [ ]:
query_ds = Dataset()

# parameters for filtering
query_ds.QueryRetrieveLevel = 'SERIES'
#query_ds.StudyDate  = '20190805-20190809'
query_ds.StudyDate  = '20190805'
query_ds.Modality = 'PT'
#query_ds.Modality = ['PT', 'CT']
#query_ds.Modality = ['PT', 'NM']

# parameters to fetch
query_ds.StudyTime  = ''
query_ds.StudyInstanceUID = ''
query_ds.SeriesTime = ''
query_ds.SeriesInstanceUID = ''
query_ds.PatientID = ''
query_ds.NumberOfSeriesRelatedInstances = ''
query_ds.StudyDescription = ''
query_ds.InstitutionName = ''
query_ds.ReferringPhysicianName = ''

# display Dataset
logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

# do the query (C-FIND)
df_all_series = find_data(config, query_ds)

# drop unwanted columns
df_all_series = df_all_series.drop(['Query/Retrieve Level', 'Retrieve AE Title',
                                    'Issuer of Patient ID', 'Type of Patient ID'],
                                   axis=1)
df_all_series['Number of Series Related Instances'] = df_all_series['Number of Series Related Instances'].astype(int)
df_all_series

Get a summary of the possible institution names
--

In [ ]:
count_inst_name = df_all_series.groupby('Institution Name')['Institution Name'].count()
count_inst_name['None'] = len(df_all_series[df_all_series['Institution Name'].isnull()])
count_inst_name

Get a summary of the possible physician names
--

In [ ]:
count_phys_name = df_all_series.groupby("Referring Physician's Name")["Referring Physician's Name"].count()
count_phys_name['None'] = len(df_all_series[df_all_series["Referring Physician's Name"].isnull()])
count_phys_name

Check the PT vs CT distribution in each institution
--

In [ ]:
for inst_name in count_inst_name.keys():
    df_inst_name = df_all_series[df_all_series['Institution Name'] == inst_name]
    count_modality = df_inst_name.groupby('Modality')['Modality'].count()
    count_PT = str(count_modality.get('PT'))
    if count_PT == 'None': count_PT = '-'
    count_CT = str(count_modality.get('CT'))
    if count_CT == 'None': count_CT = '-'
    logging.info('{:40s}: CT ={:>3s}, PT ={:>3s}'.format(inst_name, count_CT, count_PT))

Check why some series have no institution name
--

In [ ]:
start_times = df_series_updated['Study Time'].astype(float)
logging.info(start_times)
frame_dur = df_series_updated['Actual Frame Duration'].astype(float)
logging.info(frame_dur)
frame_dur = df_series_updated['Actual Frame Duration'].astype(float)
logging.info(frame_dur)

In [ ]:
df_null_inst_name = df_all_series[df_all_series['Institution Name'].isnull()].reset_index(drop=True)
#df_null_inst_name = df_all_series[df_all_series['Institution Name'] == 'PET CT CHUV'].reset_index(drop=True)
#ICD.display(df_null_inst_name)
ds = create_dataset_from_dataframe_row(df_null_inst_name.loc[4,:],
                                       exclude_columns=['Study Description', 'Number of Series Related Instances'])
ds.QueryRetrieveLevel = 'IMAGE'
to_fetch_param_names = ['StudyDescription', 'PatientID', 'InstitutionName', '0x07a31023', 'ReferringPhysicianName',
                        'SOPClassUID', 'Manufacturer', 'ManufacturerModelName',
                        'InstanceNumber', 'ContentTime', 'AcquisitionTime', 'ActualFrameDuration', 'NumberOfSlices', 
                        'PerformedProcedureStepStartDate', 'PerformedProcedureStepStartTime', 
                        'PerformedProcedureStepEndDate', 'PerformedProcedureStepEndTime',
                        'ImagesInAcquisition', 'NumberOfFrames']

df, datasets = get_data(config, ds, to_fetch_param_names)
ICD.display(df)

ds = create_dataset_from_dataframe_row(df_null_inst_name.loc[10,:],
                                       exclude_columns=['Study Description', 'Number of Series Related Instances'])
ds.QueryRetrieveLevel = 'IMAGE'
df, datasets = get_data(config, ds, to_fetch_param_names)
ICD.display(df)

Apparently, these are weird images starting with InstanceNumber 2001 or such, having no other information (AcquisitionTime, NumberOfSlices, etc.), even though they have a ContentTime. So it might be okay to leave them out.

Refine the data set based on the "Institution Name" and the "Patient ID"
--

In [ ]:
accepted_inst_names = ['chuv', 'chuvlausanne', 'centrehospitalieruniversitairevaudois', 'petctchuv',
                       'medecinenucleairechuvlausanne']
df_all_series['Short Institution Name'] = df_all_series['Institution Name'].str.lower().apply(lambda s: str(s).replace(' ', ''))
df_series = df_all_series[df_all_series['Short Institution Name'].isin(accepted_inst_names)]
df_series = df_series[df_series['Patient ID'].str.match('^\d+$')].reset_index(drop=True)
df_series
count_inst_name = df_series.groupby('Institution Name')['Institution Name'].count()
count_inst_name['None'] = len(df_series[df_series['Institution Name'].isnull()])
count_inst_name

Go through each Series and get more info
--

In [ ]:
df_series_updated = df_series.copy()

# go through each Serie
#for i_serie in range(len(df_series)):
for i_serie in range(0, 20):
    
    logging.info('{}/{}: [{}|{}|{}|PID:{}|:{}]'.format(i_serie, len(df_series), *df_series.loc[i_serie,
                ['Study Date', 'Series Time', 'Modality', 'Patient ID', 'Series Instance UID']]))
    query_ds = Dataset()
    # set all the parameters for filtering
    for col in df_series.columns:
        col_name = col.title().replace(' ', '').replace('Uid', 'UID')
        value = df_series.loc[i_serie, col]
        # avoid NaNs
        if str(value) == 'nan': value = ''
        logging.debug('Setting "{}" ("{}") to "{}" (type: "{}")'.format(col, col_name, value, type(value)))
        setattr(query_ds, col_name, value)
        
    # set the specific filtering parameters
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', df_series.loc[i_serie, 'Number of Series Related Instances']]
    # display the Dataset
    logging.debug('Query Dataset:')
    for s in str(query_ds).split('\n'): logging.debug('    ' + s)

    # fetch the data (C-MOVE)
    to_fetch_param_names = ['ManufacturerModelName', 'InstanceNumber', 'ContentTime',
                            'AcquisitionTime', 'ActualFrameDuration', 'NumberOfSlices', 
                           '0x07a31023', 'SOPClassUID' ]
    df, _ = get_data(config, query_ds, to_fetch_param_names)
    df.reset_index(drop=True, inplace=True)
    # if no data is found, skip with error
    if len(df) == 0:
        logging.error('  Problem, no data found.')
        continue
    # if there is only one row, duplicate it
    elif len(df) == 1:
        df.loc[1, :] = df.loc[0, :]
    # if there are too many rows, skip with error
    elif len(df) > 2:
        logging.error('  Problem, too many rows found ({}).'.format(len(df)))
        continue
       
    # copy the relevant parameters into the main DataFrame
    df_series_updated.loc[i_serie, 'Content Time Start']     = df.loc[0, 'ContentTime']
    df_series_updated.loc[i_serie, 'Content Time End']       = df.loc[1, 'ContentTime']
    df_series_updated.loc[i_serie, 'Acquisition Time Start'] = df.loc[0, 'AcquisitionTime']
    df_series_updated.loc[i_serie, 'Acquisition Time End']   = df.loc[1, 'AcquisitionTime']
    df_series_updated.loc[i_serie, 'Manufacturer Model Name']  = df.loc[0, 'ManufacturerModelName']
    df_series_updated.loc[i_serie, 'Actual Frame Duration']  = df.loc[0, 'ActualFrameDuration']
    df_series_updated.loc[i_serie, 'Number of Slices']  = df.loc[0, 'NumberOfSlices']
    df_series_updated.loc[i_serie, '0x07a31023']  = df.loc[0, '0x07a31023']
    df_series_updated.loc[i_serie, 'SOP Class UID']  = df.loc[0, 'SOPClassUID']
    
df_series_updated

In [ ]:
df_series_updated.loc[0:20, :]

Figure out why some data cannot be fetched

In [ ]:
df_series.loc[0]

In [ ]:
i_serie = 1

logging.info('{}/{}: [{}|{}|{}|PID:{}|:{}]'.format(i_serie, len(df_series), *df_series.loc[i_serie,
                ['Study Date', 'Series Time', 'Modality', 'Patient ID', 'Series Instance UID']]))
query_ds = Dataset()
# set all the parameters for filtering
exclude_columns = ['Number of Series Related Instances', 'Instance Number']
for col in df_series.columns:
    if col in exclude_columns: continue
    col_name = col.title().replace(' ', '').replace('Uid', 'UID')
    value = df_series.loc[i_serie, col]
    # avoid NaNs
    if str(value) == 'nan': value = ''
    logging.info('Setting "{}" ("{}") to "{}" (type: "{}")'.format(col, col_name, value, type(value)))
    setattr(query_ds, col_name, value)

# set the specific filtering parameters
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.InstanceNumber = '1'
# display the Dataset
logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

# fetch the data (C-MOVE)
to_fetch_param_names = ['ManufacturerModelName', 'InstanceNumber', 'ContentTime',
                        'AcquisitionTime', 'ActualFrameDuration']
df, datasets = get_data(config, query_ds, to_fetch_param_names)
df

In [ ]:
datasets

Get (C-MOVE) a PT series first and last frame

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.3.12.2.1107.5.6.1.2013.31330119100111184984100000023'
query_ds.PatientID = '69168'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'PT'
query_ds.InstanceNumber = ['1', '387']

df = get_data(config, query_ds, ['PatientID', 'ManufacturerModelName', 'InstanceNumber', 'StudyTime',
                                 'SeriesTime', 'ContentTime', 'AcquisitionTime'])
df

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.3.12.2.1107.5.1.4.10001.30000019100107233748400041397'
query_ds.PatientID = '1084649'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'PT'

query_ds.InstanceNumber = ['1', '1000', '2772']

df = get_data(config, query_ds, ['PatientID', 'ManufacturerModelName', 'InstanceNumber', 'StudyTime',
                                 'SeriesTime', 'ContentTime', 'AcquisitionTime'])
df

Get all the series corresponding to the query Dataset's filter [date = 2019-10-01, study time = between 8am and 9am]

In [ ]:
query_ds = Dataset()

# set the filtering parameters
query_ds.QueryRetrieveLevel = 'SERIES'
query_ds.StudyDate  = '20191001'
query_ds.SeriesDate = '20191001'
query_ds.StudyTime  = '080000-090000'
query_ds.SeriesTime = '080000-090000'

# parameters to fetch
query_ds.NumberOfSeriesRelatedInstances = ''
query_ds.SeriesInstanceUID = ''
query_ds.Modality = ''
query_ds.PatientID = ''
query_ds.StudyID = ''

logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

df_series_all = find_data(config, query_ds)
df_series_all

Get a list of Series having more than a few frames

In [ ]:
df_series = df_series_all[df_series_all['Number of Series Related Instances'].astype(int) > 2][0:3]
df_series

Get time differences by querying the ContentTime on the IMAGE level

In [ ]:
count_modality = df_series.groupby('Modality')['Modality'].count()
logging.info('Number of series per modality: NM = {NM} | PT = {PT}'.format(**count_modality))
n_instances_groupby_modality = df_series.groupby('Modality')['Number of Series Related Instances']
logging.info('Number of Instances [min]:  NM = {NM:6.1f} | PT = {PT:6.1f}'.format(**n_instances_groupby_modality.min()))
logging.info('Number of Instances [avg]:  NM = {NM:6.1f} | PT = {PT:6.1f}'.format(**n_instances_groupby_modality.mean()))
logging.info('Number of Instances [med]:  NM = {NM:6.1f} | PT = {PT:6.1f}'.format(**n_instances_groupby_modality.median()))
logging.info('Number of Instances [max]:  NM = {NM:6.1f} | PT = {PT:6.1f}'.format(**n_instances_groupby_modality.max()))


In [ ]:
time_format = '%H%M%S'

for i in df_series.index:
    
    series_UID = df_series.loc[i, 'Series Instance UID']
    last_frame = df_series.loc[i, 'Number of Series Related Instances']
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', last_frame]
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    
    start_time = dt.strptime(df.loc[0, 'Content Time'], time_format)
    end_time = dt.strptime(df.loc[1, 'Content Time'], time_format)
    delta_sec = (end_time - start_time).seconds

    logging.info('Series total time: end [{:%H:%M:%S}] - [{:%H:%M:%S}] = {} seconds'.format(end_time, start_time, delta_sec))

In [ ]:
df_series_pt = \
    df_series_all[
        (df_series_all.Modality == 'PT')
      & (df_series_all['Number of Series Related Instances'].astype(int) > 1)
    ][0:10]
ICD.display(df_series_pt)
print(df_series_pt.loc[556, 'Series Instance UID'])
print(df_series_pt.loc[556, 'Patient ID'])

In [ ]:
time_format = '%H%M%S'

for i in df_series_pt.index:
    
    series_UID = df_series_pt.loc[i, 'Series Instance UID']
    last_frame = df_series_pt.loc[i, 'Number of Series Related Instances']
    mid_frame = str(int(int(last_frame) / 2))
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', mid_frame, last_frame]
    query_ds.SeriesInstanceUID = series_UID
    query_ds.StudyTime = ''
    query_ds.SeriesTime = ''
    query_ds.ContentTime = ''
    query_ds.NumberOfSeriesRelatedInstances = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        ICD.display(df)

In [ ]:
df.loc[0, 'Content Time'].split('.')[0]

Get all PT images with InstanceNumber 1 for a specific day (20191001)

In [ ]:
query_ds = Dataset()

# set the filtering parameters
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.StudyDate  = '20191001'
query_ds.InstanceNumber = '1'
query_ds.Modality = 'PT'

# parameters to fetch
query_ds.StudyTime = ''
query_ds.SeriesTime = ''
query_ds.ContentTime = ''
query_ds.NumberOfSeriesRelatedInstances = ''
query_ds.SeriesInstanceUID = ''

logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

df_PT_IN1_20191001_all = retrieve_data(config, query_ds)
df_PT_IN1_20191001_all

Get the content time of the last frame

In [ ]:
df_PT_IN1_20191001_copy = df_PT_IN1_20191001_all.copy()
for i in df_PT_IN1_20191001_copy.index:
    
    series_UID = df_PT_IN1_20191001_copy.loc[i, 'Series Instance UID']
    last_frame = df_PT_IN1_20191001_copy.loc[i, 'Number of Series Related Instances']
    mid_frame = str(int(int(last_frame) / 2))
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = last_frame
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        df_PT_IN1_20191001_copy.loc[i, 'Last Content Time'] = df.loc[0, 'Content Time']
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = mid_frame
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        df_PT_IN1_20191001_copy.loc[i, 'Mid Content Time'] = df.loc[0, 'Content Time']

In [ ]:
df_PT_IN1_20191001_copy[['Content Time', 'Mid Content Time', 'Last Content Time', 'Instance Number', 'Number of Series Related Instances']]

In [ ]:
s = df_PT_IN1_20191001_copy.loc[0:39, 'Content Time'].apply(lambda s: s.split('.')[1]).astype(int)
e = df_PT_IN1_20191001_copy.loc[0:39, 'Last Content Time'].apply(lambda s: s.split('.')[1]).astype(int)
n = df_PT_IN1_20191001_copy.loc[0:39, 'Number of Series Related Instances'].astype(int)
d = e - s
import pandas as pd
a = pd.DataFrame([s, e, n, d, d/n]).transpose()
a.columns = ['s', 'e', 'n', 'd', 'd/n']
a[a['n'] > 400]

Try to use MOVE instead of FIND

In [ ]:
df_series.loc[0]['Series Instance UID']

In [ ]:
ds = Dataset()
ds.QueryRetrieveLevel = 'IMAGE'
ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
ds.PatientID = '2560235'
ds.StudyID = '40464'
ds.StudyDate  = '20191001'
ds.Modality = 'CT'
ds.InstanceNumber = '1'

df = retrieve_data(config, ds)
df

In [ ]:
from pydicom.dataset import Dataset

from pynetdicom import AE, evt, StoragePresentationContexts
from pynetdicom.sop_class import PatientRootQueryRetrieveInformationModelMove

datasets = []
def handle_store(event):
    """Handle a C-STORE service request"""
    print('HANDLE STORE')
    datasets.append(event.dataset)
    # Ignore the request and return Success
    return 0x0000

handlers = [(evt.EVT_C_STORE, handle_store)]

# Initialise the Application Entity
ae = AE(ae_title = config['PACS']['ae_title'])

# Add a requested presentation context
ae.add_requested_context(PatientRootQueryRetrieveInformationModelMove)

# Add the Storage SCP's supported presentation contexts
ae.supported_contexts = StoragePresentationContexts

# Start our Storage SCP in non-blocking mode, listening on port 11120
ae.ae_title = config['PACS']['ae_title']
scp = ae.start_server((config['PACS']['local_host'], config['PACS'].getint('local_port')), block=False, evt_handlers=handlers)

# Create out identifier (query) dataset
ds = Dataset()
ds.QueryRetrieveLevel = 'IMAGE'
ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
ds.PatientID = '2560235'
ds.StudyID = '40464'
ds.StudyDate  = '20191001'
ds.Modality = 'CT'
ds.InstanceNumber = ['1', '100', '200', '289', '298', '300']

# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate(config['PACS']['host'], config['PACS'].getint('port'), ae_title=config['PACS']['ae_called_title'])

if assoc.is_established:
    # Use the C-MOVE service to send the identifier
    responses = assoc.send_c_move(ds, config['PACS']['ae_title'], PatientRootQueryRetrieveInformationModelMove)

    for (status, identifier) in responses:
        if status:
            print('C-MOVE query status: 0x{0:04x}'.format(status.Status))

            # If the status is 'Pending' then `identifier` is the C-MOVE response
            if status.Status in (0xFF00, 0xFF01):
                print(identifier)
        else:
            print('Connection timed out, was aborted or received invalid response')

    # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

# Stop our Storage SCP
scp.shutdown()


In [ ]:
[ds.ContentTime for ds in datasets]

In [ ]:
datasets[0].ManufacturerModelName

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
query_ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
query_ds.PatientID = '2560235'
query_ds.StudyID = '40464'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'CT'
query_ds.InstanceNumber = ['1', '100', '131', '132']

df = find_data(config, query_ds)
df